In [3]:
import torch
import torch.nn as nn

import random

import pandas as pd
import numpy as np

import os
import sys

import hydroeval as he

import math

import time

import matplotlib.pyplot as plt

from ...SWE_Prediction_Attention.DataLoader.SDL import SpatialDataLoader

ImportError: attempted relative import with no known parent package

In [2]:
snotel_locations_info = pd.read_csv('./Data/Snotel_Locations_Filtered_v3.csv')

snotel_locations_info['Southness'] = [ math.sin(value['Slope_tif1_x']) * math.cos(value['Aspect_tif_x']) for index,value in snotel_locations_info.iterrows()]


In [3]:
sp_dataLoader = SpatialDataLoader()

sp_dataLoader.cleanDataJunk()
sp_dataLoader.prepareInputsAndOutputs()

sp_training_input , sp_training_output = sp_dataLoader.getTrainingData()
sp_testing_data_1,sp_testing_data_2, sp_testing_data_3 , sp_testing_data_4,sp_testing_data_5 = sp_dataLoader.getTestingData()

In [4]:
sp_testing_input_1 ,sp_testing_output_1 = sp_testing_data_1
sp_testing_input_2 ,sp_testing_output_2 = sp_testing_data_2
sp_testing_input_3 ,sp_testing_output_3 = sp_testing_data_3
sp_testing_input_4 ,sp_testing_output_4 = sp_testing_data_4
sp_testing_input_5 ,sp_testing_output_5 = sp_testing_data_5

In [5]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        batch_size, seq_len , embedding_dim = x.shape
        
        x = x + self.pe[:x.size(0)]
        x = self.dropout(x)
        
        seq_len , batch_size, embedding_dim = x.shape
        
        return x.reshape(batch_size,seq_len,embedding_dim)

    
class SWETransformer(nn.Module):
    def __init__(self, encoder_input_dim = 20, model_dim = 64, n_output_heads = 1,window = 3,
                  seq_length = 10):
        super().__init__()
        
        # Storing the passed argument on the class definition.
        
        self.model_dim = model_dim
        self.encoder_input_dim = encoder_input_dim
        self.n_output_heads = n_output_heads
        self.seq_length = seq_length
        
        # Linear Layers for the input.
        
        self.input_embed_1 = torch.nn.Linear(self.encoder_input_dim , int(self.model_dim/2))
        self.input_embed_2 = torch.nn.Linear(int(self.model_dim/2) , self.model_dim)

        self.input_dropout = torch.nn.Dropout(p = 0.10)
        
        # Positional Encoding
        self.positional_encoding = PositionalEncoding(self.model_dim,0.2)
           
        # Transformer model definition.
        
        self.encoder_layer_1 = nn.TransformerEncoderLayer(d_model=self.model_dim, nhead=16)
        self.transformers_1 = torch.nn.TransformerEncoder(self.encoder_layer_1 , num_layers = 24)
        
        self.encoder_layer_2 = nn.TransformerEncoderLayer(d_model=self.model_dim, nhead=8)
        self.transformers_2 = torch.nn.TransformerEncoder(self.encoder_layer_2, num_layers = 16, norm = None) 
        
        # Dimension Reduction.
        
        initial_dim  = self.model_dim
        
        self.dim_red_1 = torch.nn.Linear(initial_dim * 2 , int(initial_dim/2))
        self.dim_red_2 = torch.nn.Linear(int(initial_dim/2) , int(initial_dim/2))
        self.dim_red_3 = torch.nn.Linear(int(initial_dim/2) , int(initial_dim/4))
        self.dim_red_4 = torch.nn.Linear(int(initial_dim/4) , int(initial_dim/8))
        
        self.dim_red_dropout = torch.nn.Dropout(p = 0.05)
        
        # Final output layer for the model.
        
        self.decoder_layer_1 = torch.nn.Linear(self.seq_length * int(self.model_dim/8) ,self.seq_length * int(self.model_dim/16))    
        self.decoder_layer_2 = torch.nn.Linear(self.seq_length * int(self.model_dim/16), self.seq_length)
        
        # Activation Functions
        
        self.activation_relu = torch.nn.ReLU()
        self.activation_identity = torch.nn.Identity()
        self.activation_gelu = torch.nn.GELU()
        self.activation_tanh = torch.nn.Tanh()
        self.activation_sigmoid = torch.nn.Sigmoid()
        
        # Dropout Functions 
        
        self.dropout_5 = torch.nn.Dropout(p = 0.05)
        self.dropout_10 = torch.nn.Dropout(p = 0.10)
        self.dropout_15 = torch.nn.Dropout(p = 0.15)
        self.dropout_20 = torch.nn.Dropout(p = 0.20)
        
        
    def forward(self,encoder_inputs):
        
        # Converting to the torch array.
        
        encoder_inputs = torch.from_numpy(encoder_inputs).to(dtype= torch.float32)
                
        # Getting the configuration of the passed data.
        
        encoder_batch_size, encoder_sequence_length , encoder_input_dim = encoder_inputs.shape
                
        # Embedding the daily data passed to the model for the locations.
        
        
        embed_input_x = encoder_inputs.reshape(-1,self.encoder_input_dim)
        
        embed_input_x = self.input_embed_1(embed_input_x)
        embed_input_x = self.activation_gelu(embed_input_x)
        
        embed_input_x = self.input_embed_2(embed_input_x)
        embed_input_x = self.activation_gelu(embed_input_x)
        
        embed_input_x = embed_input_x.reshape(encoder_batch_size, encoder_sequence_length, self.model_dim)
        
        # Applying positional encoding.
        
        x = self.positional_encoding(embed_input_x)
        
        # Applying the transformer layer.

        x = self.transformers_1(embed_input_x)
        #x = self.transformers_2(x)
        
        x = x.reshape(-1, self.model_dim)
        embed_input_x = embed_input_x.reshape(-1,self.model_dim)
        
        x = torch.cat((x , embed_input_x),1)
        
        # Dim reduction layer.
        
        x = self.dim_red_1(x) 
        x= self.dropout_20(x)
        
        x = self.dim_red_2(x)
        x = self.activation_relu(x)
        x= self.dropout_20(x)

        x = self.dim_red_3(x)
        x= self.dropout_20(x)
        
        x= self.dim_red_4(x)
        x = self.activation_relu(x)
        x= self.dropout_20(x)
        
        
        
        # Final layer for the output.
        
        x = x.reshape(-1, encoder_sequence_length * int(self.model_dim/8))
        
        x= self.decoder_layer_1(x)
        x= self.activation_gelu(x)
        x = self.dropout_10(x)
        
        x = self.decoder_layer_2(x)
        x = self.activation_identity(x)
        
        x= x.reshape(encoder_batch_size , encoder_sequence_length , self.n_output_heads)
        
        return x
    


In [6]:
swe_model_sp = SWETransformer(encoder_input_dim = 19, model_dim = 512, n_output_heads = 1, seq_length = 323)


In [7]:
# Loss Function 
mean_squared_error_sp = nn.MSELoss()


In [8]:
# Optimizer
optimizer_sp = torch.optim.AdamW(swe_model_sp.parameters(), lr= 0.0001, weight_decay = 0.0001)
scheduler_sp = torch.optim.lr_scheduler.StepLR(optimizer_sp, step_size = 3 ,gamma = 0.6, last_epoch= -1, verbose=False)


In [9]:
def TrainModelSP(train_inputs, train_outputs, epoch_number, data_type= "Spatial"):
    loss_value = 0
    total_loss = 0
    total_batches = 0
    
    for input_index , batch_input in enumerate(train_inputs):
        
        total_batches +=1
        
        optimizer_sp.zero_grad()
                
        batch_size , sequence_length , feature_dim = train_outputs[input_index].shape
            
        output = swe_model_sp(batch_input)
                    
        loss = mean_squared_error_sp(output, torch.from_numpy(train_outputs[input_index]).to(dtype=torch.float32))
      
        loss_in_batch = loss.item() * batch_input.shape[0]
        
        total_loss+=loss_in_batch
        
        loss.backward()
        optimizer_sp.step()
        
        if (input_index + 1) % 1000 == 0:
            print("Obtained Output:")
            print(output )
        
            
        if  (input_index + 1) % 100 == 0:
            print('Type: {} , Epoch Number: {} => Batch number :{} , loss value : {} '.format(data_type,epoch_number,input_index+1, loss.item()))
            print("=================================================")
        
            
    print('Type:{}, Epoch Number: {} => Avg loss value : {} '.format(data_type,epoch_number, total_loss / (total_batches * 1 )))
    
    return total_loss / (total_batches * 1 )

In [ ]:
swe_model_sp.train(True)

train_epoch_avg_losses_sp = []

start_time = time.time()

for index in range(12):
    
    temp_holder_sp = list(zip(sp_training_input, sp_training_output))
    random.shuffle(temp_holder_sp)
       
    train_input_batches_sp, train_output_batches_sp = zip(*temp_holder_sp)
    
    epoch_loss_sp = TrainModelSP(train_input_batches_sp, train_output_batches_sp, epoch_number = index+ 1)
    
    train_epoch_avg_losses_sp.append(epoch_loss_sp)
    
    scheduler_sp.step()

end_time = time.time()

print("Time Elapsed", end_time - start_time)

Type: Spatial , Epoch Number: 1 => Batch number :100 , loss value : 12.171859741210938 
Type: Spatial , Epoch Number: 1 => Batch number :200 , loss value : 37.54853057861328 
Type: Spatial , Epoch Number: 1 => Batch number :300 , loss value : 17.896867752075195 
Type: Spatial , Epoch Number: 1 => Batch number :400 , loss value : 26.9755859375 
Type: Spatial , Epoch Number: 1 => Batch number :500 , loss value : 63.159183502197266 
Type: Spatial , Epoch Number: 1 => Batch number :600 , loss value : 13.87961196899414 
Type: Spatial , Epoch Number: 1 => Batch number :700 , loss value : 242.38302612304688 
Type: Spatial , Epoch Number: 1 => Batch number :800 , loss value : 7.071486949920654 
Type: Spatial , Epoch Number: 1 => Batch number :900 , loss value : 436.35101318359375 
Obtained Output:
tensor([[[ 7.0125],
         [ 7.8424],
         [ 9.5712],
         [ 2.8729],
         [ 3.0361],
         [ 8.0701],
         [12.3983],
         [ 6.3764],
         [ 8.3633],
         [ 4.0335],

Type: Spatial , Epoch Number: 1 => Batch number :1400 , loss value : 4.858253002166748 
Type: Spatial , Epoch Number: 1 => Batch number :1500 , loss value : 145.44020080566406 
Type: Spatial , Epoch Number: 1 => Batch number :1600 , loss value : 5.76478385925293 
Type: Spatial , Epoch Number: 1 => Batch number :1700 , loss value : 18.425098419189453 
Type: Spatial , Epoch Number: 1 => Batch number :1800 , loss value : 98.65473937988281 
Type: Spatial , Epoch Number: 1 => Batch number :1900 , loss value : 34.134952545166016 
Obtained Output:
tensor([[[ 0.7710],
         [ 0.1566],
         [-1.8888],
         [-0.0777],
         [ 0.4648],
         [-0.2651],
         [-0.8709],
         [ 0.2932],
         [ 1.0638],
         [ 0.7699],
         [ 1.1092],
         [ 0.5492],
         [-0.1772],
         [-0.9139],
         [-1.3063],
         [ 0.6073],
         [ 1.0961],
         [ 0.2945],
         [-0.2601],
         [-1.4907],
         [-1.7472],
         [ 0.6433],
         [-1.

Type: Spatial , Epoch Number: 1 => Batch number :2700 , loss value : 12.102312088012695 
Type: Spatial , Epoch Number: 1 => Batch number :2800 , loss value : 2.5449254512786865 
Type: Spatial , Epoch Number: 1 => Batch number :2900 , loss value : 10.712546348571777 
Obtained Output:
tensor([[[ 8.0512e+00],
         [ 1.6593e+01],
         [ 1.9024e+01],
         [ 4.7342e-01],
         [ 2.1739e+00],
         [ 1.2768e+01],
         [ 2.1669e+01],
         [ 1.1927e+01],
         [ 1.3832e+01],
         [ 7.3659e+00],
         [ 7.1510e+00],
         [ 1.4367e+01],
         [ 1.2367e+01],
         [ 1.7733e+01],
         [ 4.8499e+01],
         [ 1.7497e+01],
         [ 1.9877e+01],
         [ 1.7987e+01],
         [ 1.6163e+01],
         [ 3.2584e+01],
         [ 1.6165e+01],
         [ 5.5217e+00],
         [ 1.5894e+01],
         [ 2.6822e+01],
         [ 7.8028e+00],
         [ 1.7197e+01],
         [ 1.1434e+01],
         [ 2.3134e+01],
         [ 7.2039e+00],
         [ 3.4216e+0

Type: Spatial , Epoch Number: 1 => Batch number :3100 , loss value : 26.01458168029785 
Type: Spatial , Epoch Number: 1 => Batch number :3200 , loss value : 63.241485595703125 
Type: Spatial , Epoch Number: 1 => Batch number :3300 , loss value : 66.26860809326172 
Type: Spatial , Epoch Number: 1 => Batch number :3400 , loss value : 0.8364642262458801 
Type: Spatial , Epoch Number: 1 => Batch number :3500 , loss value : 1.155706524848938 
Type:Spatial, Epoch Number: 1 => Avg loss value : 44.496508085935716 
Type: Spatial , Epoch Number: 2 => Batch number :100 , loss value : 21.350875854492188 
Type: Spatial , Epoch Number: 2 => Batch number :200 , loss value : 81.04537200927734 
Type: Spatial , Epoch Number: 2 => Batch number :300 , loss value : 0.39514800906181335 
Type: Spatial , Epoch Number: 2 => Batch number :400 , loss value : 21.38006591796875 
Type: Spatial , Epoch Number: 2 => Batch number :500 , loss value : 3.617805004119873 
Type: Spatial , Epoch Number: 2 => Batch number :6

Type: Spatial , Epoch Number: 2 => Batch number :1100 , loss value : 15.867907524108887 
Type: Spatial , Epoch Number: 2 => Batch number :1200 , loss value : 83.03657531738281 
Type: Spatial , Epoch Number: 2 => Batch number :1300 , loss value : 0.48480573296546936 
Type: Spatial , Epoch Number: 2 => Batch number :1400 , loss value : 8.539238929748535 
Type: Spatial , Epoch Number: 2 => Batch number :1500 , loss value : 14.173965454101562 
Type: Spatial , Epoch Number: 2 => Batch number :1600 , loss value : 9.972665786743164 
Type: Spatial , Epoch Number: 2 => Batch number :1700 , loss value : 117.66429138183594 
Type: Spatial , Epoch Number: 2 => Batch number :1800 , loss value : 24.003517150878906 
Type: Spatial , Epoch Number: 2 => Batch number :1900 , loss value : 9.22841739654541 
Obtained Output:
tensor([[[ 9.1087e+00],
         [ 1.1645e+01],
         [ 1.4110e+01],
         [ 3.7194e+00],
         [ 6.7669e+00],
         [ 1.0679e+01],
         [ 1.6256e+01],
         [ 9.0070e

Type: Spatial , Epoch Number: 2 => Batch number :2100 , loss value : 2.2122626304626465 
Type: Spatial , Epoch Number: 2 => Batch number :2200 , loss value : 27.538188934326172 
Type: Spatial , Epoch Number: 2 => Batch number :2300 , loss value : 12.08181095123291 
Type: Spatial , Epoch Number: 2 => Batch number :2400 , loss value : 19.831323623657227 
Type: Spatial , Epoch Number: 2 => Batch number :2500 , loss value : 47.81752014160156 
Type: Spatial , Epoch Number: 2 => Batch number :2600 , loss value : 0.4865974485874176 
Type: Spatial , Epoch Number: 2 => Batch number :2700 , loss value : 2.785881757736206 
Type: Spatial , Epoch Number: 2 => Batch number :2800 , loss value : 5.743525981903076 
Type: Spatial , Epoch Number: 2 => Batch number :2900 , loss value : 3.9614944458007812 
Obtained Output:
tensor([[[ 3.4755e-01],
         [ 4.7529e+00],
         [ 7.6563e+00],
         [ 5.5962e-01],
         [ 8.0518e-01],
         [-1.3834e+00],
         [-9.1204e-01],
         [ 5.7543e

Type: Spatial , Epoch Number: 2 => Batch number :3100 , loss value : 0.1808536797761917 
Type: Spatial , Epoch Number: 2 => Batch number :3200 , loss value : 1.0501381158828735 
Type: Spatial , Epoch Number: 2 => Batch number :3300 , loss value : 16.27313995361328 
Type: Spatial , Epoch Number: 2 => Batch number :3400 , loss value : 15.774698257446289 
Type: Spatial , Epoch Number: 2 => Batch number :3500 , loss value : 31.83329200744629 
Type:Spatial, Epoch Number: 2 => Avg loss value : 23.126694508610118 


In [11]:
swe_model_sp.eval()

def TestModelSP(test_inputs, test_outputs):
    loss_value = 0
    losses = []
    
    mse_sp = nn.MSELoss()
    
    print("========================")
    print('Testing the model.')
    print("========================")
    
    outputs = []
    outputs_loss = []
    
    
    with torch.no_grad():
        for input_index , batch_input in enumerate(test_inputs):
            
            batch_size , sequence_length , feature_dim = test_outputs[input_index].shape
            
            #SWE_Model.transformers_1.apply(lambda module: setattr(module, "training", True))
            
            output = swe_model_sp(batch_input)
                        
            loss = mse_sp(output, torch.from_numpy(test_outputs[input_index]).to(dtype=torch.float32))
            
            outputs.append(output.detach().numpy())
            outputs_loss.append(loss.item())
            
            nse = he.evaluator(he.nse, output.detach().numpy().squeeze(), test_outputs[input_index].squeeze())
            print('Batch number :{} , loss value : {} , NSE: {} '.format(input_index+1, loss.item(), nse))
            
            losses.append(loss.item())
    
    return (np.array(outputs), np.array(outputs_loss))


In [12]:
test_outputs_1_sp, test_losses_1_sp = TestModelSP(sp_testing_input_1,sp_testing_output_1)


Testing the model.
Batch number :1 , loss value : 0.6759222745895386 , NSE: [-110.50547695] 
Batch number :2 , loss value : 2.2463529109954834 , NSE: [-144.72979328] 
Batch number :3 , loss value : 0.21075859665870667 , NSE: [-11.35113528] 
Batch number :4 , loss value : 0.26391762495040894 , NSE: [-19.00338883] 
Batch number :5 , loss value : 0.12978622317314148 , NSE: [-17.85859036] 
Batch number :6 , loss value : 0.12356973439455032 , NSE: [-48.0372986] 
Batch number :7 , loss value : 0.09527461230754852 , NSE: [-57.33961972] 
Batch number :8 , loss value : 0.22112327814102173 , NSE: [-274.88580849] 
Batch number :9 , loss value : 0.15160241723060608 , NSE: [-544.7739452] 
Batch number :10 , loss value : 0.13411998748779297 , NSE: [-58.34601858] 
Batch number :11 , loss value : 0.21800446510314941 , NSE: [-138.98146496] 
Batch number :12 , loss value : 0.4185434877872467 , NSE: [-68.64310074] 
Batch number :13 , loss value : 0.4359920620918274 , NSE: [-13.44303779] 
Batch number :14

Batch number :114 , loss value : 7.18070125579834 , NSE: [0.77268661] 
Batch number :115 , loss value : 7.363335132598877 , NSE: [0.76605492] 
Batch number :116 , loss value : 11.867874145507812 , NSE: [0.62815526] 
Batch number :117 , loss value : 12.80263900756836 , NSE: [0.60736476] 
Batch number :118 , loss value : 9.85379695892334 , NSE: [0.6998136] 
Batch number :119 , loss value : 9.214881896972656 , NSE: [0.72060616] 
Batch number :120 , loss value : 8.44503402709961 , NSE: [0.74573739] 
Batch number :121 , loss value : 7.52650260925293 , NSE: [0.77534186] 
Batch number :122 , loss value : 9.056446075439453 , NSE: [0.72991769] 
Batch number :123 , loss value : 12.261330604553223 , NSE: [0.63418611] 
Batch number :124 , loss value : 13.846163749694824 , NSE: [0.58770665] 
Batch number :125 , loss value : 18.989925384521484 , NSE: [0.45259547] 
Batch number :126 , loss value : 19.52220916748047 , NSE: [0.46443987] 
Batch number :127 , loss value : 15.829821586608887 , NSE: [0.585

Batch number :227 , loss value : 46.20307540893555 , NSE: [-0.05371289] 
Batch number :228 , loss value : 87.31880950927734 , NSE: [-1.00022879] 
Batch number :229 , loss value : 70.18290710449219 , NSE: [-0.61919421] 
Batch number :230 , loss value : 61.3819580078125 , NSE: [-0.42717356] 
Batch number :231 , loss value : 39.01798629760742 , NSE: [0.07896321] 
Batch number :232 , loss value : 32.11469650268555 , NSE: [0.23844411] 
Batch number :233 , loss value : 23.28299903869629 , NSE: [0.43371685] 
Batch number :234 , loss value : 43.689910888671875 , NSE: [-0.10050785] 
Batch number :235 , loss value : 25.04876136779785 , NSE: [0.34230794] 
Batch number :236 , loss value : 25.614164352416992 , NSE: [0.3071473] 
Batch number :237 , loss value : 23.96651840209961 , NSE: [0.32934285] 
Batch number :238 , loss value : 25.292810440063477 , NSE: [0.25945122] 
Batch number :239 , loss value : 26.283884048461914 , NSE: [0.18025347] 
Batch number :240 , loss value : 37.70821762084961 , NSE:

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/hydroeval/objective_functions.py:39: RuntimeWarning: divide by zero encountered in divide
  np.sum((evaluation - simulations) ** 2, axis=0, dtype=np.float64)


Batch number :267 , loss value : 2.4345452785491943 , NSE: [-inf] 
Batch number :268 , loss value : 2.604355573654175 , NSE: [-inf] 
Batch number :269 , loss value : 1.9035414457321167 , NSE: [-inf] 
Batch number :270 , loss value : 9.576902389526367 , NSE: [-inf] 


In [23]:
test_outputs_2_sp, test_losses_2_sp = TestModelSP(sp_testing_input_2,sp_testing_output_2)


Testing the model.
Batch number :1 , loss value : 0.820328950881958 , NSE: [-9.62214308] 
Batch number :2 , loss value : 0.3473261296749115 , NSE: [-4.69541802] 
Batch number :3 , loss value : 0.15474174916744232 , NSE: [-1.82619119] 
Batch number :4 , loss value : 0.165149986743927 , NSE: [-2.44196641] 
Batch number :5 , loss value : 0.4851553738117218 , NSE: [-9.58801066] 
Batch number :6 , loss value : 0.14449523389339447 , NSE: [-2.76351055] 
Batch number :7 , loss value : 0.2895495593547821 , NSE: [-9.25279268] 
Batch number :8 , loss value : 0.17004059255123138 , NSE: [-3.47446118] 
Batch number :9 , loss value : 0.13375769555568695 , NSE: [-1.68623048] 
Batch number :10 , loss value : 0.10504318028688431 , NSE: [-0.08450679] 
Batch number :11 , loss value : 0.19124571979045868 , NSE: [-0.68635732] 
Batch number :12 , loss value : 0.2213715761899948 , NSE: [-1.03492247] 
Batch number :13 , loss value : 0.24445044994354248 , NSE: [-1.45197357] 
Batch number :14 , loss value : 0.24

Batch number :115 , loss value : 18.785232543945312 , NSE: [0.63330941] 
Batch number :116 , loss value : 18.477285385131836 , NSE: [0.64129635] 
Batch number :117 , loss value : 19.680805206298828 , NSE: [0.61809118] 
Batch number :118 , loss value : 21.245859146118164 , NSE: [0.5873409] 
Batch number :119 , loss value : 19.828155517578125 , NSE: [0.61497354] 
Batch number :120 , loss value : 21.258811950683594 , NSE: [0.58665206] 
Batch number :121 , loss value : 22.949731826782227 , NSE: [0.55298111] 
Batch number :122 , loss value : 21.26634979248047 , NSE: [0.58647313] 
Batch number :123 , loss value : 20.097026824951172 , NSE: [0.60621773] 
Batch number :124 , loss value : 18.066530227661133 , NSE: [0.64419874] 
Batch number :125 , loss value : 19.80330467224121 , NSE: [0.60792966] 
Batch number :126 , loss value : 21.448619842529297 , NSE: [0.57484599] 
Batch number :127 , loss value : 24.929990768432617 , NSE: [0.50769615] 
Batch number :128 , loss value : 21.91986656188965 , N

Batch number :229 , loss value : 83.43214416503906 , NSE: [-0.3452564] 
Batch number :230 , loss value : 72.79306030273438 , NSE: [-0.28646447] 
Batch number :231 , loss value : 66.935791015625 , NSE: [-0.29367086] 
Batch number :232 , loss value : 56.084739685058594 , NSE: [-0.1748669] 
Batch number :233 , loss value : 59.71049499511719 , NSE: [-0.29706327] 
Batch number :234 , loss value : 59.29927062988281 , NSE: [-0.29970559] 
Batch number :235 , loss value : 56.25149917602539 , NSE: [-0.2556285] 
Batch number :236 , loss value : 47.89822769165039 , NSE: [-0.10420619] 
Batch number :237 , loss value : 53.707672119140625 , NSE: [-0.28887325] 
Batch number :238 , loss value : 50.75697708129883 , NSE: [-0.28894491] 
Batch number :239 , loss value : 29.798704147338867 , NSE: [0.17542918] 
Batch number :240 , loss value : 42.248538970947266 , NSE: [-0.26330587] 
Batch number :241 , loss value : 34.53446960449219 , NSE: [-0.10934394] 
Batch number :242 , loss value : 37.0280647277832 , N

In [27]:
test_outputs_3_sp, test_losses_3_sp = TestModelSP(sp_testing_input_3,sp_testing_output_3)


Testing the model.
Batch number :1 , loss value : 1.5700100660324097 , NSE: [-3.51700368] 
Batch number :2 , loss value : 0.8191308975219727 , NSE: [-0.95763159] 
Batch number :3 , loss value : 0.38414934277534485 , NSE: [0.29280991] 
Batch number :4 , loss value : 0.4956668019294739 , NSE: [0.13235179] 
Batch number :5 , loss value : 0.4488498568534851 , NSE: [0.23649006] 
Batch number :6 , loss value : 0.4474078118801117 , NSE: [0.21057611] 
Batch number :7 , loss value : 0.583640992641449 , NSE: [-0.04274254] 
Batch number :8 , loss value : 0.8811006546020508 , NSE: [-0.55203533] 
Batch number :9 , loss value : 1.0199617147445679 , NSE: [-0.65422883] 
Batch number :10 , loss value : 0.7279695868492126 , NSE: [-0.16040215] 
Batch number :11 , loss value : 0.6356063485145569 , NSE: [0.00738625] 
Batch number :12 , loss value : 0.5356008410453796 , NSE: [0.18809849] 
Batch number :13 , loss value : 0.7238470315933228 , NSE: [0.01892297] 
Batch number :14 , loss value : 0.76991128921508

Batch number :116 , loss value : 19.984067916870117 , NSE: [0.56877946] 
Batch number :117 , loss value : 28.00777244567871 , NSE: [0.41493351] 
Batch number :118 , loss value : 28.680927276611328 , NSE: [0.41786655] 
Batch number :119 , loss value : 26.262399673461914 , NSE: [0.49058843] 
Batch number :120 , loss value : 27.228727340698242 , NSE: [0.49651543] 
Batch number :121 , loss value : 26.02777671813965 , NSE: [0.53306306] 
Batch number :122 , loss value : 23.365833282470703 , NSE: [0.59283285] 
Batch number :123 , loss value : 26.637928009033203 , NSE: [0.54294552] 
Batch number :124 , loss value : 27.054662704467773 , NSE: [0.54025262] 
Batch number :125 , loss value : 24.647682189941406 , NSE: [0.59324682] 
Batch number :126 , loss value : 32.367427825927734 , NSE: [0.50644484] 
Batch number :127 , loss value : 28.515640258789062 , NSE: [0.57725206] 
Batch number :128 , loss value : 32.44721984863281 , NSE: [0.55143839] 
Batch number :129 , loss value : 31.348405838012695 , 

Batch number :231 , loss value : 80.36749267578125 , NSE: [0.43589869] 
Batch number :232 , loss value : 92.30933380126953 , NSE: [0.32636349] 
Batch number :233 , loss value : 92.880126953125 , NSE: [0.2873011] 
Batch number :234 , loss value : 112.07405090332031 , NSE: [0.09338583] 
Batch number :235 , loss value : 99.65386962890625 , NSE: [0.14075144] 
Batch number :236 , loss value : 81.59280395507812 , NSE: [0.25913998] 
Batch number :237 , loss value : 85.21920776367188 , NSE: [0.16477998] 
Batch number :238 , loss value : 85.59005737304688 , NSE: [0.08828677] 
Batch number :239 , loss value : 103.5322265625 , NSE: [-0.19744629] 
Batch number :240 , loss value : 85.48690795898438 , NSE: [-0.07070247] 
Batch number :241 , loss value : 67.38536071777344 , NSE: [0.08353406] 
Batch number :242 , loss value : 66.24938201904297 , NSE: [0.01318944] 
Batch number :243 , loss value : 75.19831848144531 , NSE: [-0.21420603] 
Batch number :244 , loss value : 75.24494934082031 , NSE: [-0.2824

In [29]:
test_outputs_4_sp, test_losses_4_sp = TestModelSP(sp_testing_input_4,sp_testing_output_4)


Testing the model.
Batch number :1 , loss value : 0.47498464584350586 , NSE: [-90.06301661] 
Batch number :2 , loss value : 0.4303506314754486 , NSE: [-110.82577291] 
Batch number :3 , loss value : 0.21085986495018005 , NSE: [-24.71755649] 
Batch number :4 , loss value : 0.12222039699554443 , NSE: [-0.43405857] 
Batch number :5 , loss value : 0.21079877018928528 , NSE: [-0.23156501] 
Batch number :6 , loss value : 0.21400600671768188 , NSE: [0.00849646] 
Batch number :7 , loss value : 0.21291117370128632 , NSE: [0.03608806] 
Batch number :8 , loss value : 0.22005414962768555 , NSE: [0.01799617] 
Batch number :9 , loss value : 0.25785696506500244 , NSE: [-0.34741152] 
Batch number :10 , loss value : 0.19149863719940186 , NSE: [-0.18615098] 
Batch number :11 , loss value : 0.292034387588501 , NSE: [-0.87044135] 
Batch number :12 , loss value : 0.7889833450317383 , NSE: [-3.94212964] 
Batch number :13 , loss value : 1.6459002494812012 , NSE: [-9.18627524] 
Batch number :14 , loss value : 

Batch number :115 , loss value : 24.916078567504883 , NSE: [0.59738069] 
Batch number :116 , loss value : 26.306543350219727 , NSE: [0.57754283] 
Batch number :117 , loss value : 26.918773651123047 , NSE: [0.56864999] 
Batch number :118 , loss value : 26.868759155273438 , NSE: [0.57060159] 
Batch number :119 , loss value : 26.08158302307129 , NSE: [0.58265385] 
Batch number :120 , loss value : 24.377925872802734 , NSE: [0.61093043] 
Batch number :121 , loss value : 31.765743255615234 , NSE: [0.49398738] 
Batch number :122 , loss value : 32.68916320800781 , NSE: [0.48009935] 
Batch number :123 , loss value : 21.619609832763672 , NSE: [0.6542443] 
Batch number :124 , loss value : 24.69924545288086 , NSE: [0.60601189] 
Batch number :125 , loss value : 41.732887268066406 , NSE: [0.34903743] 
Batch number :126 , loss value : 45.42104721069336 , NSE: [0.33155285] 
Batch number :127 , loss value : 49.21198654174805 , NSE: [0.30592006] 
Batch number :128 , loss value : 39.7005729675293 , NSE: 

Batch number :229 , loss value : 97.645263671875 , NSE: [0.62117826] 
Batch number :230 , loss value : 100.25454711914062 , NSE: [0.60690743] 
Batch number :231 , loss value : 89.9229507446289 , NSE: [0.64129246] 
Batch number :232 , loss value : 88.8690414428711 , NSE: [0.63808066] 
Batch number :233 , loss value : 79.44657897949219 , NSE: [0.66744764] 
Batch number :234 , loss value : 76.40269470214844 , NSE: [0.66908891] 
Batch number :235 , loss value : 88.72419738769531 , NSE: [0.59573636] 
Batch number :236 , loss value : 82.32064819335938 , NSE: [0.60509322] 
Batch number :237 , loss value : 72.03313446044922 , NSE: [0.64126003] 
Batch number :238 , loss value : 61.7254753112793 , NSE: [0.68204016] 
Batch number :239 , loss value : 62.62522888183594 , NSE: [0.66583693] 
Batch number :240 , loss value : 64.40016174316406 , NSE: [0.63844695] 
Batch number :241 , loss value : 48.99462127685547 , NSE: [0.70840622] 
Batch number :242 , loss value : 50.116085052490234 , NSE: [0.684306

In [31]:
test_outputs_5_sp, test_losses_5_sp = TestModelSP(sp_testing_input_5,sp_testing_output_5)

Testing the model.
Batch number :1 , loss value : 1.1659876108169556 , NSE: [-75.67784024] 
Batch number :2 , loss value : 0.40128013491630554 , NSE: [-7.84514865] 
Batch number :3 , loss value : 0.45956289768218994 , NSE: [-3.60991811] 
Batch number :4 , loss value : 0.22820529341697693 , NSE: [-0.35470386] 
Batch number :5 , loss value : 0.17937695980072021 , NSE: [0.19121012] 
Batch number :6 , loss value : 0.39995670318603516 , NSE: [-0.18391239] 
Batch number :7 , loss value : 0.44429290294647217 , NSE: [-0.11049597] 
Batch number :8 , loss value : 0.4033142626285553 , NSE: [0.02318871] 
Batch number :9 , loss value : 0.41772040724754333 , NSE: [-0.16036608] 
Batch number :10 , loss value : 0.2925649583339691 , NSE: [0.08000934] 
Batch number :11 , loss value : 0.33087971806526184 , NSE: [-0.2556864] 
Batch number :12 , loss value : 0.3050481975078583 , NSE: [-0.33003856] 
Batch number :13 , loss value : 0.4328306317329407 , NSE: [-0.99941322] 
Batch number :14 , loss value : 0.42

Batch number :115 , loss value : 9.85047435760498 , NSE: [0.80102986] 
Batch number :116 , loss value : 8.37389087677002 , NSE: [0.83175229] 
Batch number :117 , loss value : 8.730396270751953 , NSE: [0.82376937] 
Batch number :118 , loss value : 10.88548755645752 , NSE: [0.78264669] 
Batch number :119 , loss value : 17.9126033782959 , NSE: [0.66411246] 
Batch number :120 , loss value : 12.971114158630371 , NSE: [0.77265513] 
Batch number :121 , loss value : 16.938865661621094 , NSE: [0.71362281] 
Batch number :122 , loss value : 23.629863739013672 , NSE: [0.63571322] 
Batch number :123 , loss value : 27.28024673461914 , NSE: [0.60542135] 
Batch number :124 , loss value : 32.23878479003906 , NSE: [0.58281237] 
Batch number :125 , loss value : 34.746681213378906 , NSE: [0.58104481] 
Batch number :126 , loss value : 47.151248931884766 , NSE: [0.46139566] 
Batch number :127 , loss value : 51.23560333251953 , NSE: [0.42347244] 
Batch number :128 , loss value : 54.413150787353516 , NSE: [0.

Batch number :229 , loss value : 101.75293731689453 , NSE: [0.62670499] 
Batch number :230 , loss value : 107.0804443359375 , NSE: [0.59425696] 
Batch number :231 , loss value : 96.36148071289062 , NSE: [0.62123815] 
Batch number :232 , loss value : 94.63023376464844 , NSE: [0.60977406] 
Batch number :233 , loss value : 121.81793975830078 , NSE: [0.47502215] 
Batch number :234 , loss value : 129.15333557128906 , NSE: [0.42934143] 
Batch number :235 , loss value : 143.6684112548828 , NSE: [0.36273167] 
Batch number :236 , loss value : 135.9013214111328 , NSE: [0.39307042] 
Batch number :237 , loss value : 140.24766540527344 , NSE: [0.36301166] 
Batch number :238 , loss value : 121.88580322265625 , NSE: [0.43654085] 
Batch number :239 , loss value : 117.38750457763672 , NSE: [0.44344044] 
Batch number :240 , loss value : 113.39166259765625 , NSE: [0.44443742] 
Batch number :241 , loss value : 120.32682037353516 , NSE: [0.39240244] 
Batch number :242 , loss value : 71.00833892822266 , NSE